In [1]:
!pip install pdfminer.six
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 4.7 MB/s eta 0:00:00


In [12]:
import pypdf
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTLine, LTFigure
from PIL import Image
import matplotlib.pyplot as plt
import os

In [4]:
# Отображение картинки pyplot
def plt_img_show(img):
  fig = plt.figure(dpi=4)
  plt.imshow(img)
  plt.axis('off')
  plt.show()

# Классы Голосования

In [9]:
class Point:
  'Точка с координатами'

  def __init__(self, x:float, y:float):
    self.x: float = round(x,2)
    self.y: float = round(y,2)

#____________________________________________________________________
class Marker:
  'Маркер'
  def __init__(self, name: str, x0: float, y0: float, x1: float, y1: float):
    self.name: str = name
    self.cornerLD: Point = Point(x0, y0)
    self.cornerRUp: Point = Point(x1, y1)

  def getCorners(self):
    return [self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]
#____________________________________________________________________
class WordData:
  'Данные слова голосования'
  def __init__(self, word: str, cornerLD: Point, cornerRUp: Point):
    self.word: str = word
    self.cornerLD: Point = cornerLD
    self.cornerRUp: Point = cornerRUp

  def getCorners(self):
    return [self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]

#____________________________________________________________________
class LineIssue:
  'Строка данных по вопросу'
  def __init__(self, issue: str):
    self.issue: str = issue # Номер вопроса
    self.words: dict[int, WordData] = {}

  def addWord(self, wordLineNumber: int, word: str, cornerLD: Point, cornerRUp: Point):
    self.words[wordLineNumber] = WordData(word, cornerLD, cornerRUp)

#____________________________________________________________________
class BallotNumber:
  'Номер голосования'
  def __init__(self, value: str, x0: float, y0: float, x1: float, y1: float):
    self.value: str = value
    self.cornerLD: Point = Point(x0, y0)
    self.cornerRUp: Point = Point(x1, y1)

  def getCorners(self):
    return [self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]

#____________________________________________________________________
class Footer:
  'Нижний колонтитул'
  def __init__(self, pageNumber: str, ballotNumber: str, x0: float, y0: float, x1: float, y1: float):
    self.pageNumber: str = pageNumber
    self.ballotNumber: str = ballotNumber
    self.cornerLD: Point = Point(x0, y0)
    self.cornerRUp: Point = Point(x1, y1)

  def getCorners(self):
    return [self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]

#____________________________________________________________________
class BallotPage:
  'Страница бюллетеня'
  def __init__(self):
    self.cornerLD: Point
    self.cornerRD: Point
    self.cornerLUp: Point
    self.cornerRUp: Point
    self.images = {}
    self.__lineIdDict: dict[int, int] = {} # Словарь id номеров строк голосования lineId:number
    self.linesOfVoices: dict[int, LineIssue] = {}
    self.footer: Footer
    self.markers: dict[int, Marker] = {}

  def setCorners(self, cornerLD: Point, cornerRUp: Point):
    self.cornerLD = cornerLD
    self.cornerRUp = cornerRUp
    self.cornerRD = Point(cornerRUp.x, cornerLD.y)
    self.cornerLUp = Point(cornerLD.x, cornerRUp.y)

  def setCoordinates(self, x0: float, y0: float, x1: float, y1: float):
    self.cornerLD = Point(x0, y0)
    self.cornerRUp = Point(x1, y1)
    self.cornerRD = Point(x1, y0)
    self.cornerLUp = Point(x0, y1)

  def setImages(self, imageName: str, image):
    self.images[imageName] = image

  def setIssue(self, lineId: int, issue: str, wordLineNumber: int, word: str, cornerLD: Point, cornerRUp: Point):
    if lineId not in self.__lineIdDict:
      self.__lineIdDict[lineId] = len(self.__lineIdDict)
      self.linesOfVoices[self.__lineIdDict[lineId]] = LineIssue(issue=issue)
    self.linesOfVoices[self.__lineIdDict[lineId]].addWord(wordLineNumber=wordLineNumber, word=word, cornerLD=cornerLD, cornerRUp=cornerRUp)

  def setFooter(self, pageNumber: str, ballotNumber: str, x0: float, y0: float, x1: float, y1: float):
    self.footer = Footer(pageNumber, ballotNumber, x0, y0, x1, y1)

  def setMarker(self, markerName: str, x0: float, y0: float, x1: float, y1: float):
    self.markers[len(self.markers)] = Marker(markerName, x0, y0, x1, y1)

  def getCorners(self):
    return [self.cornerLD.x, self.cornerLD.y], [self.cornerRUp.x, self.cornerRUp.y]

  def getImages(self):
    result = []
    for key, img in self.images.items():
      result.append([key, img])
    return result

  def getLinesOfVoices(self):
    result = []
    for key, lineIssue in self.linesOfVoices.items():
      result.append([lineIssue.issue, lineIssue.words])
    return result

  def getMarkers(self):
    result = []
    for key, marker in self.markers.items():
      img=None
      for ikey, image in self.images.items():
        if marker.name == ikey.rsplit(".", 1)[0]:
          img=image
          break
      result.append([key, marker, img])
    return result
#____________________________________________________________________
class BallotPaper:
  'Бюллетень голосования'
  def __init__(self, fileName, pageCount):
    self.fileName: str = fileName
    self.ballotNumber: BallotNumber
    self.ballotPages: list[BallotPage] = []
    for i in range(pageCount):
      self.ballotPages.append(BallotPage())

  def setBallotNumber(self, value: str, x0: float, y0: float, x1: float, y1: float):
    self.ballotNumber = BallotNumber(value, x0, y0, x1, y1)

  def setPageCorners(self, pageNum: int, cornerLD: Point, cornerRUp: Point):
    self.ballotPages[pageNum].setCorners(cornerLD, cornerRUp)

  def setPageCoordinates(self, pageNum: int, x0: float, y0: float, x1: float, y1: float):
    self.ballotPages[pageNum].setCoordinates(x0, y0, x1, y1)

  def setPageImage(self, pageNum: int, imageName: str, image):
    self.ballotPages[pageNum].setImages(imageName, image)

  def setPageIssue(self, pageNum: int, lineId: int, issue:str, wordLineNumber: int, word: str, x0: float, y0: float, x1: float, y1: float):
    self.ballotPages[pageNum].setIssue(lineId=lineId, issue=issue, wordLineNumber=wordLineNumber, word=word, cornerLD=Point(x0, y0), cornerRUp=Point(x1, y1))

  def setPageFooter(self, pageNum: int, pageNumber: str, ballotNumber: str, x0: float, y0: float, x1: float, y1: float):
    self.ballotPages[pageNum].setFooter(pageNumber, ballotNumber, x0, y0, x1, y1)

  def getCorners(self):
    result = []
    for page_num, ballot_page in enumerate(self.ballotPages):
      result.append([page_num, ballot_page.getCorners()])
    return result

  def getBullotNumber(self):
    return self.ballotNumber

  def getData(self):
    result = []
    for page_num, ballot_page in enumerate(self.ballotPages):
      result.append([page_num, ballot_page.getCorners(), ballot_page.getImages(), ballot_page.getMarkers(), ballot_page.getLinesOfVoices(), ballot_page.footer])
    return result

#____________________________________________________________________
class Ballot:
  'Голосование'

  wordsOfVoices = ['ЗА','ПРОТИВ', 'ВОЗДЕРЖАЛСЯ'] # список слов голосования
  wordsVLines: dict[int, list[int]] = {0: [0,3], 1:[3, 6], 2:[6, 8]} # номера вертикальных линий для определения бокса каждого слова
  line_pixels_range = 10 # Диапазон по у, в пределах которого слова голосования wordsOfVoices принимаются находящимися в одной строке

  def getData(self):
    return self.baseBallotPaper.getData()

  def getVoicesLines(self, pageElements, wordIndex: int, yMax:float, yMin: float):
    vLines: dict[int, list[float]] = {}
    ln_pixels_range = 4  # Диапазон по x, в пределах которого вертикальные линии таблиц голосования lines принимаются одинаковыми
    # Перебираем элементы, составляющие страницу сверху вниз
    for i, component in enumerate(pageElements):
      # Извлекаем элемент структуры страницы
      element = component[1]
      y = element.y1
      if (y>yMin and y<yMax) and isinstance(element, LTLine): # Ищем линии в диапазоне от вопроса голосования до строки голосования
        p0=Point(element.x0, element.y0)
        p1=Point(element.x1, element.y1)
        if abs(p1.x-p0.x) < 1: # Если координаты x элемента равны (вертикальная линия)
          key = int(p0.x/ln_pixels_range+0.5)
          line = vLines.get(key, None)
          x=p0.x
          if line is not None:
            x = Point((line[0]+x)/2,0).x # Округляем через Point
          vLines[key] = [x, p0.y, p1.y]
    # Отсортируем словарь по ключу
    vLines = dict(sorted(vLines.items()))
    # заменим ключи на порядковые номера
    count = len(vLines)
    key_list = list(range(count))
    vLines = dict(zip(key_list, list(vLines.values())))
    return [int(yMin), vLines]

  def __init__(self, baseFileName):
    # создаём объект файла PDF
    pdfFile = open(baseFileName, 'rb')

    # Используем pypdf для определения размеров страницы и извлечения образцов картинок
    # создаём объект считывателя PDF
    pdfReaded = pypdf.PdfReader(pdfFile)
    pages = pdfReaded.pages
    self.baseBallotPaper: BallotPaper = BallotPaper(baseFileName, len(pages))
    for pageNum, page in enumerate(pages) :
      #Определяем координаты углов страницы
      box = page.mediabox
      self.baseBallotPaper.setPageCoordinates(pageNum = pageNum, x0 = box[0], y0 = box[1], x1 = box[2], y1 = box[3])

      # Определяем имеющиеся на странице картинки
      for j, img in enumerate(page.images):
        self.baseBallotPaper.setPageImage(pageNum=pageNum, imageName = img.name, image=img.image) # если на странице может быть несколько разных картинок с одинаковым именем (не уверен), то будет только последняя

    # Используем pdfminer.six для работы с объектами PDF
    prevTextData = {'text':'_', 'y':None} # Задаем словарь для хранения предыдущего текста и его координаты y
    vLines = None # Переменная для промежуточного хранения словаря вертикальных линий строки
    for pageNum, page in enumerate(extract_pages(baseFileName)):
      # Находим все элементы, добавляем координату y1 и элемент
      pageElements = [(element.y1, element) for element in page._objs]
      # Сортируем все элементы по порядку нахождения на странице
      pageElements.sort(key=lambda a: a[0], reverse=True) # Сортировка по убыванию, так как y=0 - нижний край страницы

      # Перебираем элементы, составляющие страницу сверху вниз
      for i, component in enumerate(pageElements):
        # Извлекаем элемент структуры страницы
        element = component[1]

        if element.y1>800 or element.y1<50 and isinstance(element, LTFigure):
          name = element.name
          pageImages = self.baseBallotPaper.ballotPages[pageNum].getImages()
          for i, image_data in enumerate(pageImages):
            if name == image_data[0].rsplit(".", 1)[0]:
              box = element.bbox
              self.baseBallotPaper.ballotPages[pageNum].setMarker(markerName=name, x0=box[0], y0=box[1], x1=box[2], y1=box[3])
              break
          #print(element.name, element.bbox) [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1]

        if isinstance(element, LTTextContainer):
          text = ''.join(c for c in element.get_text() if c.isprintable())
          if element.y1>750 and ' №' in text:
            texts = text.split(sep='№')
            value = '' if len(texts)<2 else texts[1]
            self.baseBallotPaper.setBallotNumber(value=value, x0=element.x0, y0=element.y0, x1=element.x1, y1=element.y1)
          elif element.y1>100: # Не включаем тексты, координаты которых заведомо выходят за пределы размещения текстовых блоков с вопросами
            try:
              index = self.wordsOfVoices.index(text) # Определяем индекс слова из списка слов голосования
              line_id = int(element.y0/self.line_pixels_range + 0.5) # Определяем id строки с учетом координаты y в пределах +-line_pixels_range/2
              if vLines is None: # Если линии еще не считывали
                if prevTextData['y']>element.y0:
                  yMax = prevTextData['y']
                else:
                  yMax = self.baseBallotPaper.ballotPages[pageNum].getCorners()[1][1]
                vLines = self.getVoicesLines(pageElements=pageElements, wordIndex=index, yMax=yMax, yMin=element.y0) # Находим вертикальные линии таблицы строки голосования
              x0=y0=x1=y1=0
              if abs(element.y0-vLines[0])<self.line_pixels_range:
                left, right = self.wordsVLines.get(index, [0,1])
                x0=vLines[1][left][0]
                y0=vLines[1][left][1]
                x1=vLines[1][right][0]
                y1=vLines[1][right][2]
              self.baseBallotPaper.setPageIssue(
                  pageNum=pageNum, lineId=line_id, issue=prevTextData['text'],
                    wordLineNumber=index, word=text, x0=x0, y0=y0, x1=x1, y1=y1)
            except ValueError:
              # Сохраняем текст и координату y для последующих итераций (когда будет найдена строка голосования)
              prevTextData['text'] = '' if len(text.split())<1 else text.split()[0]
              prevTextData['y'] = element.y0
              vLines = None
          elif element.y1<50 and '№' in text:
            texts = text.split()
            footerPageNumber = '' if len(texts)<1 else texts[0]
            footerBallotNumber = '' if len(texts)<2 else texts[1].split('№')[1]
            self.baseBallotPaper.setPageFooter(pageNum=pageNum, pageNumber=footerPageNumber,
              ballotNumber=footerBallotNumber, x0=element.x0, y0=element.y0, x1=element.x1, y1=element.y1)



# Создание голосования и вывод данных

In [11]:
#ballot = Ballot('Bulletin_Садовый_ул_Садовая,_29_meeting_No_1.pdf')
ballot = Ballot('помещение 1, ул. Садовая, 29 Садовый - собрание №1 - БЮЛЛЕТЕНЬ №6391122-3690641-5274319-1-71.pdf')
print(f'Имя базового PDF-файла: "{ballot.baseBallotPaper.fileName}"')
ballot_number = ballot.baseBallotPaper.getBullotNumber()
print(f'№ бюллетеня:{ballot_number.value}')
print(f'Координаты номера бюллетеня: cornerLD={ballot_number.getCorners()[0]}, cornerRUp={ballot_number.getCorners()[1]}')
ballot_data = ballot.getData()
for page_num, page_corners, page_images, page_markers_data, page_lines, footer in ballot_data:
  print(f'  Page №{page_num +1}')
  print(f'    cornerLD={page_corners[0]}, cornerRUp={page_corners[1]}')
  # for i, image_data in enumerate(page_images):
  #   print(f'      image №{i+1}: name={image_data[0]} type={type(image_data[1])}')
  #   plt_img_show(image_data[1])
  for i, page_marker_data in enumerate(page_markers_data):
    marker = page_marker_data[1]
    print(f'      Маркер №{i+1}: name={marker.name}, cornerLD={marker.getCorners()[0]}, cornerRUp={marker.getCorners()[1]}')
    plt_img_show(page_marker_data[2])
  for i, page_line in enumerate(page_lines):
    print(f'      line №{i+1}: issue="{page_line[0]}"')
    for j, word_data in page_line[1].items():
      print(f'        word №{j}: "{word_data.word}", cornerLD={word_data.getCorners()[0]}, cornerRUp={word_data.getCorners()[1]}')
  print(f'    Нижний колонтитул: номер страницы: {footer.pageNumber}, № бюллетеня: {footer.ballotNumber}, cornerLD={footer.getCorners()[0]}, cornerRUp={footer.getCorners()[1]}')



Имя базового PDF-файла: "помещение 1, ул. Садовая, 29 Садовый - собрание №1 - БЮЛЛЕТЕНЬ №6391122-3690641-5274319-1-71.pdf"
№ бюллетеня:6391122-3690641-5274319/1-71
Координаты номера бюллетеня: cornerLD=[42.52, 775.79], cornerRUp=[427.66, 792.29]
  Page №1
    cornerLD=[0, 0], cornerRUp=[595.28, 841.89]
      Маркер №1: name=I3, cornerLD=[42.52, 808.88], cornerRUp=[50.02, 816.38]


      Маркер №2: name=I4, cornerLD=[545.26, 808.88], cornerRUp=[552.76, 816.38]


      Маркер №3: name=I4, cornerLD=[42.52, 25.51], cornerRUp=[50.02, 33.01]


      Маркер №4: name=I4, cornerLD=[545.26, 25.51], cornerRUp=[552.76, 33.01]


      line №1: issue="1.1"
        word №0: "ЗА", cornerLD=[50.48, 378.37], cornerRUp=[201.62, 393.28]
        word №1: "ПРОТИВ", cornerLD=[201.62, 378.37], cornerRUp=[379.31, 393.28]
        word №2: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 378.37], cornerRUp=[510.23, 393.28]
      line №2: issue="2.1"
        word №0: "ЗА", cornerLD=[50.48, 325.61], cornerRUp=[201.62, 340.53]
        word №1: "ПРОТИВ", cornerLD=[201.62, 325.61], cornerRUp=[379.31, 340.53]
        word №2: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 325.61], cornerRUp=[510.23, 340.53]
      line №3: issue="3."
        word №0: "ЗА", cornerLD=[50.48, 285.09], cornerRUp=[201.62, 300.0]
        word №1: "ПРОТИВ", cornerLD=[201.62, 285.09], cornerRUp=[379.31, 300.0]
        word №2: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 285.09], cornerRUp=[510.23, 300.0]
      line №4: issue="4."
        word №0: "ЗА", cornerLD=[50.48, 207.88], cornerRUp=[201.62, 222.79]
        word №1: "ПРОТИВ", cornerLD=[201.62, 207.88], cornerRUp=[379.31, 222.79]
        wor

...

      line №1: issue="6."
        word №0: "ЗА", cornerLD=[50.48, 781.32], cornerRUp=[201.66, 796.22]
        word №1: "ПРОТИВ", cornerLD=[201.66, 781.32], cornerRUp=[379.38, 796.22]
        word №2: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.38, 781.32], cornerRUp=[510.25, 796.22]
      line №2: issue="7."
        word №0: "ЗА", cornerLD=[50.48, 728.57], cornerRUp=[201.62, 743.49]
        word №1: "ПРОТИВ", cornerLD=[201.62, 728.57], cornerRUp=[379.31, 743.49]
        word №2: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 728.57], cornerRUp=[510.23, 743.49]
      line №3: issue="8."
        word №0: "ЗА", cornerLD=[50.48, 663.59], cornerRUp=[201.62, 678.5]
        word №1: "ПРОТИВ", cornerLD=[201.62, 663.59], cornerRUp=[379.31, 678.5]
        word №2: "ВОЗДЕРЖАЛСЯ", cornerLD=[379.31, 663.59], cornerRUp=[510.23, 678.5]
      line №4: issue="9."
        word №0: "ЗА", cornerLD=[50.48, 598.61], cornerRUp=[201.62, 613.52]
        word №1: "ПРОТИВ", cornerLD=[201.62, 598.61], cornerRUp=[379.31, 613.52]
        word 

      Маркер №2: name=I4, cornerLD=[545.26, 808.88], cornerRUp=[552.76, 816.38]


      Маркер №3: name=I4, cornerLD=[42.52, 25.51], cornerRUp=[50.02, 33.01]


      Маркер №4: name=I4, cornerLD=[545.26, 25.51], cornerRUp=[552.76, 33.01]


    Нижний колонтитул: номер страницы: 3, № бюллетеня: 6391122-3690641-5274319/1-71, cornerLD=[164.98, 26.81], cornerRUp=[430.3, 37.81]
